In [1]:
import json
from StringIO import StringIO
import pandas as pd

In [2]:
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, names
import datetime
from scipy.stats import ttest_ind 
import csv

In [3]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [4]:
def string_to_set(x):
    c = set()
    for w in x[1:-1].split(","):
        c.add(w)
        
    return c

def has_amenity(x, amen_):
    if amen_ in x:
        return 1
    pass

In [5]:
a = listings['amenities'].fillna('{}')

listings['amenities_set'] = a.map(string_to_set)
all_amenities = set()
    
for idx in listings['amenities_set'].index:
    all_amenities = all_amenities.union(listings['amenities_set'][idx])

In [6]:
b = set(['', '"translation missing: en.hosting_amenity_49"','"translation missing: en.hosting_amenity_50"'])

In [7]:
all_amenities = all_amenities.difference(b)

In [8]:
def boolean_convert(z):
    if z:
        return 1
    else:
        return 0

In [9]:
listings['amenities_set'].map(lambda x: '"24-Hour Check-in"' in x).map(boolean_convert)

0       0
1       0
2       0
3       1
4       1
5       0
6       0
7       0
8       0
9       1
10      0
11      1
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      1
23      0
24      0
25      0
26      0
27      1
28      0
29      0
       ..
6578    0
6579    0
6580    0
6581    0
6582    0
6583    0
6584    0
6585    1
6586    0
6587    0
6588    0
6589    1
6590    0
6591    0
6592    0
6593    0
6594    1
6595    1
6596    0
6597    0
6598    0
6599    0
6600    0
6601    0
6602    1
6603    0
6604    0
6605    0
6606    1
6607    0
Name: amenities_set, dtype: int64

In [10]:
has_amenities_list = []
for amen in all_amenities:
    listings['has_' + amen] = listings['amenities_set'].map(lambda x: amen in x).map(boolean_convert)
    has_amenities_list.append('has_' + amen)

In [11]:
listings[has_amenities_list].isnull().sum()

has_"Other pet(s)"                  0
has_Essentials                      0
has_"Carbon Monoxide Detector"      0
has_"Elevator in Building"          0
has_"Indoor Fireplace"              0
has_Internet                        0
has_Washer                          0
has_Hangers                         0
has_"Buzzer/Wireless Intercom"      0
has_TV                              0
has_Gym                             0
has_"Fire Extinguisher"             0
has_"Hot Tub"                       0
has_"Cable TV"                      0
has_"Air Conditioning"              0
has_"Suitable for Events"           0
has_Kitchen                         0
has_"Family/Kid Friendly"           0
has_Shampoo                         0
has_Heating                         0
has_"Hair Dryer"                    0
has_Cat(s)                          0
has_"Smoke Detector"                0
has_Iron                            0
has_"Free Parking on Premises"      0
has_"Pets live on this property"    0
has_"Safety 

In [12]:
listings[has_amenities_list].sum()

has_"Other pet(s)"                    71
has_Essentials                      5547
has_"Carbon Monoxide Detector"      4170
has_"Elevator in Building"          1063
has_"Indoor Fireplace"              1695
has_Internet                        4756
has_Washer                          5192
has_Hangers                         3414
has_"Buzzer/Wireless Intercom"       338
has_TV                              5274
has_Gym                              974
has_"Fire Extinguisher"             3114
has_"Hot Tub"                       1458
has_"Cable TV"                      3365
has_"Air Conditioning"              2914
has_"Suitable for Events"            639
has_Kitchen                         6088
has_"Family/Kid Friendly"           4140
has_Shampoo                         4364
has_Heating                         5193
has_"Hair Dryer"                    3008
has_Cat(s)                           339
has_"Smoke Detector"                5385
has_Iron                            2915
has_"Free Parkin

In [13]:
a = listings['amenities'].fillna('{}')

listings['amenities_set'] = a.map(string_to_set)
all_amenities = set()
    
for idx in listings['amenities_set'].index:
    all_amenities = all_amenities.union(listings['amenities_set'][idx])
    
b = set(['', '"translation missing: en.hosting_amenity_49"','"translation missing: en.hosting_amenity_50"'])

for amen in all_amenities:
    
    if len(amen.split(' ')) == 1:
        listings['has_' + amen] = listings['amenities_set'].map(lambda x: amen in x).map(boolean_convert)
        continue
        
    if "" in amen:
        amen = amen[1:-1].replace(' ', '_')
            
    listings['has_' + amen] = listings['amenities_set'].map(lambda x: amen in x).map(boolean_convert)
    


In [14]:
a = listings['host_verifications'].map(lambda x: x[1:-1]).map(lambda j: j.split(',')).map(lambda k: set(k))
all_host_verifications = set()
    
for w in a.index:
    all_host_verifications = all_host_verifications.union(a[w])

In [15]:
for w in all_host_verifications:
    
    b = a.map(lambda x: w in x).map(boolean_convert)
    
    if '' in w:
        w = w.strip()[1:-1].replace(' ', '_')
            
    listings['uses_' + w] = b
    

In [16]:
uses_verification_list = []
for veri in all_host_verifications:
    
    if '' in veri:
        veri = veri.strip()[1:-1].replace(' ', '_')
            
    uses_verification_list.append('uses_' + veri)

In [17]:
listings[uses_verification_list]

,uses_phone,uses_linkedin,uses_email,uses_manual_online,uses_facebook,uses_amex,uses_phone,uses_sent_id,uses_jumio,uses_google,uses_weibo,uses_manual_offline,uses_kba,uses_kba,uses_reviews
0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1
1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1
2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1
4,0,0,1,0,1,0,0,0,0,0,0,0,1,1,1
5,0,0,1,0,1,0,0,0,0,0,0,0,1,1,1
6,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1
7,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1
8,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1
9,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1


In [18]:
listings[uses_verification_list].isnull().sum()

uses_phone             0
uses_linkedin          0
uses_email             0
uses_manual_online     0
uses_facebook          0
uses_amex              0
uses_phone             0
uses_sent_id           0
uses_jumio             0
uses_google            0
uses_weibo             0
uses_manual_offline    0
uses_kba               0
uses_kba               0
uses_reviews           0
dtype: int64

In [19]:
listings[uses_verification_list].sum()

uses_phone              186
uses_linkedin           113
uses_email             6421
uses_manual_online      102
uses_facebook          1354
uses_amex                68
uses_phone              186
uses_sent_id              7
uses_jumio             2211
uses_google             166
uses_weibo                1
uses_manual_offline     128
uses_kba               2756
uses_kba               2756
uses_reviews           5706
dtype: int64

In [20]:
a = listings['host_verifications'].map(lambda x: x[1:-1]).map(lambda j: j.split(',')).map(lambda k: set(k))
all_host_verifications = set()
    
for w in a.index:
    all_host_verifications = all_host_verifications.union(a[w])
    
for w in all_host_verifications:
    
    b = a.map(lambda x: w in x).map(boolean_convert)
    
    if '' in w:
        w = w.strip()[1:-1].replace(' ', '_')
            
    listings['uses_' + w] = b
    

In [21]:
listings[listings['uses_weibo'] == 1]

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,uses_manual_online,uses_facebook,uses_amex,uses_sent_id,uses_jumio,uses_google,uses_weibo,uses_manual_offline,uses_kba,uses_reviews
6246,5523515,https://www.airbnb.com/rooms/5523515,20160706203047,2016-07-07,Sunny Room/Private Bath in UTC UCSD,"You will fall in love - UTC, UCSD. A walkable ...",LOCATION! LOCATION! LOCATION! Remodeled SUPER ...,"You will fall in love - UTC, UCSD. A walkable ...",none,"The University Towne Centre (UTC) area, locate...",...,0,0,0,0,1,0,1,0,0,1


In [23]:
listings[has_amenities_list].sum().index

Index([u'has_"Other pet(s)"', u'has_Essentials',
       u'has_"Carbon Monoxide Detector"', u'has_"Elevator in Building"',
       u'has_"Indoor Fireplace"', u'has_Internet', u'has_Washer',
       u'has_Hangers', u'has_"Buzzer/Wireless Intercom"', u'has_TV',
       u'has_Gym', u'has_"Fire Extinguisher"', u'has_"Hot Tub"',
       u'has_"Cable TV"', u'has_"Air Conditioning"',
       u'has_"Suitable for Events"', u'has_Kitchen',
       u'has_"Family/Kid Friendly"', u'has_Shampoo', u'has_Heating',
       u'has_"Hair Dryer"', u'has_Cat(s)', u'has_"Smoke Detector"',
       u'has_Iron', u'has_"Free Parking on Premises"',
       u'has_"Pets live on this property"', u'has_"Safety Card"',
       u'has_"Smoking Allowed"', u'has_Dryer', u'has_Pool',
       u'has_"Pets Allowed"', u'has_"Wheelchair Accessible"',
       u'has_"First Aid Kit"', u'has_Dog(s)', u'has_"Wireless Internet"',
       u'has_"Laptop Friendly Workspace"', u'has_Breakfast', u'has_Doorman',
       u'has_"Lock on Bedroom Door"', u'h

In [25]:
high_quality_amenities_list = []
mid_quality_amenities_list = []
low_quality_amenities_list = []


for w in listings[has_amenities_list].sum().index:
    a = listings[has_amenities_list].sum()[w]
    
    if a < 2000:
        high_quality_amenities_list.append(w)
        
    elif (2000 <= a) and (a < 3400):
        mid_quality_amenities_list.append(w)
        
    else:
        low_quality_amenities_list.append(w)

In [26]:
high_quality_amenities_list

['has_"Other pet(s)"',
 'has_"Elevator in Building"',
 'has_"Indoor Fireplace"',
 'has_"Buzzer/Wireless Intercom"',
 'has_Gym',
 'has_"Hot Tub"',
 'has_"Suitable for Events"',
 'has_Cat(s)',
 'has_"Pets live on this property"',
 'has_"Safety Card"',
 'has_"Smoking Allowed"',
 'has_Pool',
 'has_"Pets Allowed"',
 'has_"Wheelchair Accessible"',
 'has_Dog(s)',
 'has_Breakfast',
 'has_Doorman',
 'has_"Lock on Bedroom Door"']

In [27]:
mid_quality_amenities_list

['has_"Fire Extinguisher"',
 'has_"Cable TV"',
 'has_"Air Conditioning"',
 'has_"Hair Dryer"',
 'has_Iron',
 'has_"First Aid Kit"',
 'has_"Laptop Friendly Workspace"',
 'has_"24-Hour Check-in"']

In [28]:
low_quality_amenities_list

['has_Essentials',
 'has_"Carbon Monoxide Detector"',
 'has_Internet',
 'has_Washer',
 'has_Hangers',
 'has_TV',
 'has_Kitchen',
 'has_"Family/Kid Friendly"',
 'has_Shampoo',
 'has_Heating',
 'has_"Smoke Detector"',
 'has_"Free Parking on Premises"',
 'has_Dryer',
 'has_"Wireless Internet"']

In [29]:
def listings_quality_classification(listings):
    listings['amenity_level'] = ''
    
    
    for w in listings.index:
        
        for j in high_quality_amenities_list:
            a = listings[j][w]
            
            if a == 1:
                listings['amenity_level'][w] = 'high'
                break 
                
        if listings['amenity_level'][w] == 'high':
            continue
                
        for k in mid_quality_amenities_list:
            a = listings[k][w]
            
            if a == 1:
                listings['amenity_level'][w] = 'mid_level'
                break
                
        if listings['amenity_level'][w] == 'mid_level':
            continue
            
        else:
            listings['amenity_level'][w] = 'low'
            
    return listings 

In [30]:
listings_quality_classification(listings)

C:\Users\sanka\Anaconda2\envs\py27\lib\site-packages\ipykernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sanka\Anaconda2\envs\py27\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sanka\Anaconda2\envs\py27\lib\site-packages\ipykernel\__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,uses_facebook,uses_amex,uses_sent_id,uses_jumio,uses_google,uses_weibo,uses_manual_offline,uses_kba,uses_reviews,amenity_level
0,11204286,https://www.airbnb.com/rooms/11204286,20160706203047,2016-07-07,Family friendly/California king,"Aquatica Waterpark, Sleep train Amphitheater, ...","Walking to Aquatica Waterpark, Sleep train Amp...","Aquatica Waterpark, Sleep train Amphitheater, ...",none,NaN,...,0,0,0,1,0,0,0,0,1,mid_level
1,7972006,https://www.airbnb.com/rooms/7972006,20160706203047,2016-07-07,Welcome to Sunset Suite,Your spacious room awaiting is with a Queen Si...,NaN,Your spacious room awaiting is with a Queen Si...,none,Getting around is easy. Very close to Eastlake...,...,0,0,0,1,0,0,0,0,1,high
2,7727710,https://www.airbnb.com/rooms/7727710,20160706203047,2016-07-07,San Diego/Eastlake. Gated community,"This is an immaculate 3 bedroom, 2 1/2 bath co...",NaN,"This is an immaculate 3 bedroom, 2 1/2 bath co...",none,NaN,...,0,0,0,0,0,0,0,0,1,high
3,13124681,https://www.airbnb.com/rooms/13124681,20160706203047,2016-07-07,Townhome in Eastlake,This 2 Story TownHome is close to Otay Ranch ...,"My place is good for couples, business travele...",This 2 Story TownHome is close to Otay Ranch ...,none,"Located in eastern Chula Vista, Otay Ranch is ...",...,0,0,0,1,0,0,0,0,1,high
4,3469225,https://www.airbnb.com/rooms/3469225,20160706203047,2016-07-07,Bedroom suite in Large new home,Hello; we are offering a private secluded bedr...,"Beautiful, quiet award-winning suburban neighb...",Hello; we are offering a private secluded bedr...,none,"The quiet serenity; near Park and lakes, beaut...",...,1,0,0,0,0,0,0,1,1,high
5,877473,https://www.airbnb.com/rooms/877473,20160706203047,2016-07-07,Private Bedroom in Newer Home,"Private Bedroom with full size bed, + futon, p...","Private bedroom with full size Bed, in large n...","Private Bedroom with full size bed, + futon, p...",none,"It is quiet, serene, tranquil, suburban; yet c...",...,1,0,0,0,0,0,0,1,1,high
6,3124507,https://www.airbnb.com/rooms/3124507,20160706203047,2016-07-07,"Great view to mountains,lake,trails","In this beautiful home, located near the Olymp...",The location and view of lake and mountain,"In this beautiful home, located near the Olymp...",none,Like countryside and also near from downtown S...,...,0,0,0,0,0,0,0,1,1,high
7,3432507,https://www.airbnb.com/rooms/3432507,20160706203047,2016-07-07,2 bedrooms with shared bath,In a great house with view on Otay Lakes and ...,"the proximity to lake ,trail and Olympic train...",In a great house with view on Otay Lakes and ...,none,Freedom in nature,...,0,0,0,0,0,0,0,1,1,high
8,3249729,https://www.airbnb.com/rooms/3249729,20160706203047,2016-07-07,Near to Olympic Training center,Beautiful home in the Eastlake community with ...,Location on the border of the town with nice t...,Beautiful home in the Eastlake community with ...,none,"open , quiet, country feeling",...,0,0,0,0,0,0,0,1,1,high
9,11756336,https://www.airbnb.com/rooms/11756336,20160706203047,2016-07-07,Chula Vista Golf Course House,"Feel right at home in our spacious 5BR, 3.5BA ...","Feel right at home in our spacious 5BR, 3.5BA ...","Feel right at home in our spacious 5BR, 3.5BA ...",none,Our home is surrounded by natural beauty. Visi...,...,0,0,0,1,0,0,0,0,1,mid_level


In [37]:
high_quality_amenities_list_two = []

for w in high_quality_amenities_list:
    a = w.split('_')[1][1:-1].split(' ')
    b = w.split('_')[0]
    
    if len(a) > 1:
        q = b + '_'
        
        for j in a:
            p = j + '_'
            q = q + p
            
        q = q[:-1]
        high_quality_amenities_list_two.append(q)
        continue
        
    high_quality_amenities_list_two.append(w)
    

In [38]:
high_quality_amenities_list_two

['has_Other_pet(s)',
 'has_Elevator_in_Building',
 'has_Indoor_Fireplace',
 'has_Buzzer/Wireless_Intercom',
 'has_Gym',
 'has_Hot_Tub',
 'has_Suitable_for_Events',
 'has_Cat(s)',
 'has_Pets_live_on_this_property',
 'has_Safety_Card',
 'has_Smoking_Allowed',
 'has_Pool',
 'has_Pets_Allowed',
 'has_Wheelchair_Accessible',
 'has_Dog(s)',
 'has_Breakfast',
 'has_Doorman',
 'has_Lock_on_Bedroom_Door']

In [39]:
mid_quality_amenities_list_two = []

for w in mid_quality_amenities_list:
    a = w.split('_')[1][1:-1].split(' ')
    b = w.split('_')[0]
    
    if len(a) > 1:
        q = b + '_'
        
        for j in a:
            p = j + '_'
            q = q + p
            
        q = q[:-1]
        mid_quality_amenities_list_two.append(q)
        continue
        
    mid_quality_amenities_list_two.append(w)
    

In [41]:
mid_quality_amenities_list_two

['has_Fire_Extinguisher',
 'has_Cable_TV',
 'has_Air_Conditioning',
 'has_Hair_Dryer',
 'has_Iron',
 'has_First_Aid_Kit',
 'has_Laptop_Friendly_Workspace',
 'has_24-Hour_Check-in']

In [42]:
low_quality_amenities_list_two = []

for w in low_quality_amenities_list:
    a = w.split('_')[1][1:-1].split(' ')
    b = w.split('_')[0]
    
    if len(a) > 1:
        q = b + '_'
        
        for j in a:
            p = j + '_'
            q = q + p
            
        q = q[:-1]
        low_quality_amenities_list_two.append(q)
        continue
        
    low_quality_amenities_list_two.append(w)
    

In [43]:
low_quality_amenities_list_two

['has_Essentials',
 'has_Carbon_Monoxide_Detector',
 'has_Internet',
 'has_Washer',
 'has_Hangers',
 'has_TV',
 'has_Kitchen',
 'has_Family/Kid_Friendly',
 'has_Shampoo',
 'has_Heating',
 'has_Smoke_Detector',
 'has_Free_Parking_on_Premises',
 'has_Dryer',
 'has_Wireless_Internet']